In [27]:
from typing import Literal

import numpy as np
from pprint import pprint

import sklearn

In [3]:
type Sample = list[int]
type Data = list[Sample]

type Label = int
type Labels = list[Label]

In [16]:
# TODO: add more funcs

from abc import ABC, abstractmethod



class ActivationBase(ABC):
    @abstractmethod
    def calc(self, x: Sample) -> list[float]:
        """Apply the activation function to an input"""
        pass


class ReLU(ActivationBase):
    def calc(self, x) -> list[float]:
        return np.maximum(0, x)


class ELU(ActivationBase):
    def __init__(self, alpha: float = 1.0) -> None:
        self.alpha = alpha
    
    def calc(self, x: Sample) -> list[float]:
        return np.where(x > 0, x, self.alpha * (np.exp(x) - 1))


class Sigmoid(ActivationBase):
    def calc(self, x: Sample) -> list[float]:
        return 1 / (1 + np.exp(-x))


class Tahn(ActivationBase):
    def calc(self, x: Sample) -> list[float]:
        return np.tanh(x)


class Softmax(ActivationBase):
    def calc(self, x: Sample) -> list[float]:

        max_value = np.max(x)
        x -= max_value

        exp_values = np.exp(x)
        return exp_values / np.sum(exp_values)

In [6]:
a = [1, 100, 22, 99]

f = Softmax()
f.calc(a)

array([7.39262147e-44, 7.31058579e-01, 9.74950551e-35, 2.68941421e-01])

In [8]:
class Dataset:
    def __init__(self, data: Data, labels: Labels) -> None:
        self.data: Data = data
        self._len = len(data)
        self.labels: Labels = labels

    def __len__(self) -> int:
        return self._len
    
    def __getitem__(self, index) -> Sample:
        return self.data[index]
    
    def __iter__(self):
        return iter(self.data)


In [17]:
class Layer:
    def __init__(self, n_inputs: int, n_neurons: int, activation: ActivationBase) -> None:
        self.n_inputs = n_inputs
        self.n_neurons = n_neurons
        
        self.weights = self._init_weights()
        self.biases = self._init_biases()

        self.activation = activation
    
    def _init_weights(self) -> list[float]:
        return np.random.randn(self.n_inputs, self.n_neurons) * 0.1
    
    def _init_biases(self):
        return np.random.randn(self.n_inputs, self.n_neurons)
    
    def forward(self, inputs) -> None:
        # print(f"{inputs = }")
        # print(f"{self.weights = }")
        # print(f"{self.biases = }")
        
        output = np.dot(inputs, self.weights)

        # print(output)

        # output += self.biases
        output = self.biases + output



        self.output = self.activation.calc(output)
        print(f"{output = }")


Layers = list[Layer]

In [18]:
class NeuralNetwork:
    def __init__(self, layers: Layers):
        self.layers = layers
        self._layers_len = len(layers)

    def fit(self, dataset: Dataset):
        for i,sample in enumerate(dataset):
            print("_______ Layer", 1, '\n\n')
            self.layers[0].forward(inputs=sample)
            
            for i in range(1, self._layers_len):
                print("______ Layer", i+1, '\n\n')
                self.layers[i].forward(inputs=self.layers[i-1].output)
    
    def predict(self):
        ...
    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights.append(layer.weights)
        return weights

In [19]:
train_data = [
    [1,2,3,4],
    # [4,3,2,1]
]


# train_data = [
#     [
#         [1,2,3,4],
#         [1,2,3,4],
#     ],
# ]

val_data = [
    [1,2,3,4],
    [4,3,2,1]
]


train_dataset = Dataset(data=train_data, labels=[1,1])
# val_dataset = Dataset(data=val_data)

In [23]:
layers = [
    Layer(4,1, activation=ReLU()),
    Layer(1,1, activation=ReLU()),
    Layer(1,5, activation=Softmax()),
]

model = NeuralNetwork(layers=layers)

In [21]:
a = np.array([[1,2,3],[1,2,3],])

b = np.array([[1,2,3],[1,2,3],])


# np.dot(a,b.T)
a + [1]

array([[2, 3, 4],
       [2, 3, 4]])

In [24]:
model.fit(dataset=train_dataset)

_______ Layer 1 


output = array([[-0.31398245],
       [ 1.18054483],
       [-0.90531665],
       [ 0.12439545]])
______ Layer 2 


output = array([[-0.38354324],
       [-0.29798873],
       [-0.38354324],
       [-0.37452826]])
______ Layer 3 


output = array([[-0.03876571,  0.        , -1.42527781, -2.70225199, -1.70905178],
       [-0.03876571,  0.        , -1.42527781, -2.70225199, -1.70905178],
       [-0.03876571,  0.        , -1.42527781, -2.70225199, -1.70905178],
       [-0.03876571,  0.        , -1.42527781, -2.70225199, -1.70905178]])


In [66]:
model.layers[-1].

array([[-1.6845608 ,  1.45302676],
       [ 2.22517856,  0.78169763],
       [ 0.97735487, -0.91557542],
       [-1.6375041 , -0.2103463 ]])

In [208]:
model.weights

[array([[ 0.04662128,  0.00362523,  0.11101263,  0.21666945],
        [-0.0427738 , -0.02047442,  0.00652623,  0.12543406],
        [ 0.04504755, -0.13728407,  0.14258488, -0.1006611 ],
        [-0.21504161, -0.09286219,  0.02443559, -0.06513455]]),
 array([[ 0.01747982,  0.05444045],
        [ 0.05884424, -0.05411848],
        [-0.07776638,  0.02290638],
        [ 0.02842925,  0.12456388]])]